<a href="https://colab.research.google.com/github/zfukuoka/Copying_a_sutra/blob/master/SoftwareDesign202010_01/SoftwareDesign202010_0104.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SoftwareDesign 2020年10月号第一特集 Pythonではじめる統計学

公式のサンプルコードの写経

## 第4章 未知のデータを知るための推測統計

In [2]:
# 基本ライブラリ
import pandas as pd
import numpy as np
import numpy.random as rd
import scipy.stats as st
from tqdm import tqdm
from IPython.display import display

# グラフ描画系
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation as ani
import seaborn as sns

# plot settings
plt.rcParams["patch.force_edgecolor"] = True
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})

%config InlineBackend.figure_format='retina'

In [3]:
!pip install japanize_matplotlib

     |████████████████████████████████| 4.1MB 2.8MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-cp36-none-any.whl size=4120275 sha256=a3be7929394f2f6f9890154e37cfea4737448bc634dd0380b291ce0c385754e2
  Stored in directory: /root/.cache/pip/wheels/b7/d9/a2/f907d50b32a2d2008ce5d691d30fb6569c2c93eefcfde55202
Successfully built japanize-matplotlib


In [4]:
import japanize_matplotlib

＃ ランダムサンプリングによる標本抽出

In [6]:
from scipy.stats import norm

In [7]:
# コード17: 身長データの母集団を生成

np.random.seed(71)
n_data = 50000000

# 日本人男性5000万人の身長を模したデータを生成。これを母集団とする。
# 母集団 171.4cm, 母標準偏差 5.8cm
height_men = norm.rvs(loc=171.4, scale=5.8, size=n_data)
population_mean = np.mean(height_men)
population_std = np.std(height_men)
print(f"母集団: {population_mean:.1f}, 母標準偏差: {population_std:.1f}")

母集団: 171.4, 母標準偏差: 5.8
